<a href="https://colab.research.google.com/github/rupaut98/nbody_research/blob/main/nbody_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y octave

In [ ]:
# In a Colab cell
# This enables Octave (MATLAB-like) syntax

# Install Octave magic for Colab
!pip install oct2py

# Load Octave magic
%load_ext oct2py.ipython


In [8]:
%%octave

% Function Definitions

function [x, converged] = newton_method(x0, max_iter, tol, lb, ub)
    x = x0;
    converged = false;

    for iter = 1:max_iter
        F = myfun(x);
        J = jacobian_num(x);

        if rcond(J) < eps
            break;
        end

        delta = -J \ F;

        alpha = 1;

        while true
            x_new = x + alpha * delta;

            if any(x_new < lb) || any(x_new > ub)
                alpha = alpha / 2;
                if alpha < 1e-4
                    break;
                end
                continue;
            end

            F_new = myfun(x_new);

            if norm(F_new) < norm(F) || alpha < 1e-4
                break;
            end

            alpha = alpha / 2;
        end

        x = x_new;

        if norm(delta) < tol
            converged = true;
            break;
        end
    end
end

function J = jacobian_num(x)
    epsilon = 1e-6;
    n = length(x);
    F0 = myfun(x);
    m = length(F0);
    J = zeros(m, n);

    for i = 1:n
        x_eps_plus = x;
        x_eps_minus = x;
        x_eps_plus(i) = x_eps_plus(i) + epsilon;
        x_eps_minus(i) = x_eps_minus(i) - epsilon;
        F_plus = myfun(x_eps_plus);
        F_minus = myfun(x_eps_minus);
        J(:, i) = (F_plus - F_minus) / (2 * epsilon);
    end
end

function F = myfun(x)
    F_original = compute_residuals(x);
    F = F_original;
end

function F_original = compute_residuals(x)
    x3 = x(1);
    x4 = x(2);
    y3 = x(3);
    y4 = x(4);

    m1 = 5;
    m2 = 4;
    m3 = 3;
    m4 = 5;

    term_a = ((-1 - x3)^2 + y3^2)^(-3/2);
    term_b = ((1 - x3)^2 + y3^2)^(-3/2);
    term_c = ((-1 - x4)^2 + y4^2)^(-3/2);
    term_d = ((1 - x4)^2 + y4^2)^(-3/2);
    term_e = ((x3 - x4)^2 + (y3 - y4)^2)^(-3/2);
    term_f = ((x3 - 1)^2 + y3^2)^(-3/2);

    f12 = 2 * m3 * (term_a - term_b) * y3 + 2 * m4 * (term_c - term_d) * y4;
    term_constant = 1/8;
    f13 = -2 * m2 * (term_constant - term_b) * y3 + ...
          m4 * (term_c - term_e) * ((x4 + 1) * (y4 - y3) + y3 * (x3 - x4));
    f24 = 2 * m1 * (term_constant - term_c) * y4 + ...
          m3 * (term_f - term_e) * (-y3 * (1 - x4) - y4 * (x3 - 1));
    f34 = m1 * (term_a - term_c) * ((x4 + 1) * (y4 - y3) + y3 * (x3 - x4)) + ...
          m2 * (term_b - term_d) * (y3 * (1 - x4) + y4 * (x3 - 1));

    F_original = [f12; f13; f24; f34];
end

function f34 = compute_f34(x)
    x3 = x(1);
    x4 = x(2);
    y3 = x(3);
    y4 = x(4);

    m1 = 5;
    m2 = 4;
    m3 = 3;
    m4 = 5;

    term_a = ((-1 - x3)^2 + y3^2)^(-3/2);
    term_b = ((1 - x3)^2 + y3^2)^(-3/2);
    term_c = ((-1 - x4)^2 + y4^2)^(-3/2);
    term_d = ((1 - x4)^2 + y4^2)^(-3/2);
    term_e = ((x3 - x4)^2 + (y3 - y4)^2)^(-3/2);
    term_f = ((x3 - 1)^2 + y3^2)^(-3/2);

    f34 = m1 * (term_a - term_c) * ((x4 + 1) * (y4 - y3) + y3 * (x3 - x4)) + ...
          m2 * (term_b - term_d) * (y3 * (1 - x4) + y4 * (x3 - 1));
end


In [ ]:
%%octave

% Main script logic
clear;
clc;

lb = [0; -1; 1.73; 0];
ub = [1; 0; 3.73; 1.73];

num_divisions = [20, 20, 20, 20];

x3_vals = linspace(lb(1), ub(1), num_divisions(1));
x4_vals = linspace(lb(2), ub(2), num_divisions(2));
y3_vals = linspace(lb(3), ub(3), num_divisions(3));
y4_vals = linspace(lb(4), ub(4), num_divisions(4));

[X3_grid, X4_grid, Y3_grid, Y4_grid] = ndgrid(x3_vals, x4_vals, y3_vals, y4_vals);
grid_guesses = [X3_grid(:), X4_grid(:), Y3_grid(:), Y4_grid(:)];

num_random_guesses = 10000;
random_guesses = lb' + rand(num_random_guesses, 4) .* (ub' - lb');

initial_guesses = [grid_guesses; random_guesses];

max_iter = 100;
tol = 1e-8;

num_guesses = size(initial_guesses, 1);
solutions = NaN(4, num_guesses);
f34_values = NaN(1, num_guesses);
converged_flags = false(1, num_guesses);

for i = 1:num_guesses
    x0 = initial_guesses(i, :)';
    [x_sol, converged] = newton_method(x0, max_iter, tol, lb, ub);
    if converged
        f34 = compute_f34(x_sol);
        if abs(f34) < tol
            solutions(:, i) = x_sol;
            f34_values(i) = f34;
            converged_flags(i) = true;
        end
    end
end

valid_indices = converged_flags;
solutions = solutions(:, valid_indices);
f34_values = f34_values(valid_indices);

fprintf('Number of converged solutions: %d\n', size(solutions, 2));
